<a href="https://colab.research.google.com/github/elenagonzalezrebollo/AprendizajeAutomatico/blob/main/Practica3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
data = pd.read_csv('vehcle_img_data.csv', encoding='latin-1')
data

,Image_Path,Brand,Model,Edition,Year,Condition,Transmission,Body,Fuel,Capacity
0,vehicle_images/1633889154186_Toyota Hilux 2006...,Toyota,Hilux,Hilux,2006,Used,Manual,SUV / 4x4,Diesel,"2,500 cc"
1,vehicle_images/1633889154395_Land Rover Freela...,Land Rover,Freelander,1999,1999,Used,Manual,SUV / 4x4,Diesel,"1,990 cc"
2,vehicle_images/1633889154545_Nissan Leaf G Gra...,Nissan,Leaf,G Grade,2014,Used,Automatic,Hatchback,Electric,70 cc
3,vehicle_images/1633889154697_Mercedes Benz C20...,Mercedes Benz,C200,C-Cabriolet AMG Line,2016,Used,Automatic,Convertible,Petrol,"2,000 cc"
4,vehicle_images/1633889154870_Toyota Vitz 2018 ...,Toyota,Vitz,2018,2018,Used,Automatic,Hatchback,Petrol,"1,000 cc"
...,...,...,...,...,...,...,...,...,...,...
77975,./vehicle_images/1636063962173_Honda Insight Z...,Honda,Insight Ze3,NaN,2012,Used,Automatic,Car,Hybrid,1500
77976,./vehicle_images/1636063962320_HINO Dutro 2011...,HINO,Dutro,NaN,2011,Recondition,Manual,Unregistered,Diesel,4000
77977,./vehicle_images/1636063962474_TVS King 2017 T...,TVS,King,NaN,2017,Used,Manual,Three Wheel,Petrol,90
77978,./vehicle_images/1636063962690_Toyota Vitz 201...,Toyota,Vitz,NaN,2018,Used,Automatic,Car,Petrol,1000


In [49]:
import zipfile
with zipfile.ZipFile('vehicle_images.zip','r') as zip_ref: 
  zip_ref.extractall('.')

In [53]:
# Pintamos la tercera imagen de entrenamiento (la número 2 en el array)
import matplotlib.pyplot as plt
digit = images[1]
plt.imshow(digit, cmap=plt.cm.binary)
plt.show()

NameError: ignored